# Prueba Valorización Depósitos a Plazo

In [1]:
import QC_Financial as qcf

Un depósito a plazo en CLP o USD se modela como un simple cashflow. Este a su vez se construye con un monto, una fecha y una moneda.

In [2]:
# Con estas variables vamos a construir
fecha_vcto = qcf.QCDate(12, 1, 2019)
monto = 10000000.0
clp = qcf.QCCLP()

# Se construye el depósito
depo = qcf.SimpleCashflow(fecha_vcto, monto, clp)

In [3]:
# Chequeo
print depo.amount()
print depo.date()

10000000.0
12-1-2019


In [4]:
# Ahora damos de alta una curva. Construimos un vector de plazos (enteros) y 
# un vector de tasas (flotantes).

plazos = qcf.long_vec()
tasas = qcf.double_vec()
for i in range(0, 12):
    plazos.append(30 + i * 30)
    tasas.append(.001 + i * .002)

crv = qcf.QCCurve(plazos, tasas)

In [5]:
# Chequeo
for i in range(0, 12):
    print crv.get_values_at(i).tenor, crv.get_values_at(i).value

30 0.001
60 0.003
90 0.005
120 0.007
150 0.009
180 0.011
210 0.013
240 0.015
270 0.017
300 0.019
330 0.021
360 0.023


In [6]:
# A la curva le vamos a agregar un interpolador (lineal)
crv_lin = qcf.QCLinearInterpolator(crv)

In [7]:
# Chequeo
crv_lin.interpolate_at(370)

0.023

In [8]:
# Se establece la convención de las tasas para finalmente construir una curva cero cupón.
# El valor de la tasa es irrelevante, sólo nos interesan las convenciones.
yf = qcf.QCAct30()
wf = qcf.QCLinearWf()
tasa = qcf.QCInterestRate(0.0, yf, wf)

In [9]:
# Se construye la curva definitiva
z_crv_lin = qcf.ZeroCouponCurve(crv_lin, tasa)

In [10]:
# Chequeo
plazo = 12
tasa = z_crv_lin.get_rate_at(plazo)
print tasa
print z_crv_lin.get_discount_factor_at(plazo)
print "check df:", 1 / (1 + tasa * plazo / 30)

0.001
0.999600159936
check df: 0.999600159936


In [19]:
# Nuevo método
qc_rate = z_crv_lin.get_qc_interest_rate_at(370)
print qc_rate.get_value()
print 1/qc_rate.wf(370)
print z_crv_lin.get_discount_factor_at(370)

0.023
0.77901843677
0.77901843677


In [ ]:
# Se construye un objeto PresentValue para valorizar
valorizador = qcf.PresentValue()

In [ ]:
# Valorizamos el depo.
# Establecemos la fecha de valorización
fecha_val = qcf.QCDate(31, 12, 2018)

# Calculamos el valor
valor = valorizador.pv(fecha_val, depo, z_crv_lin)
print "valor:", valor
print "check:", 10000000.0 * 0.999600159936

# Tenemos también las derivadas
for d in valorizador.get_derivatives():
    print d * .0001